In [1]:
import pymongo
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from PIL import Image, ImageTk
from pymongo import MongoClient
import re

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017")
mydb = myclient["Pharmacy"]
mycol = mydb["Patient"]


In [3]:
lst = ["PID", "Full Name", "Sex", "Address", "Contact", "Patient Details"]

def display_all_records():
    def sort_records_by_id():
        tree.delete(*tree.get_children())
        for record in mycol.find().sort("PID"):
            pid = record.get("PID", "")
            name = record.get("Name", "")
            sex = record.get("Sex", "")
            address = record.get("Address", "")
            contact = record.get("Contact", "")
            patient_details = record.get("Patient Details", "")
            tree.insert('', 'end', values=(pid, name, sex, address, contact, patient_details))

    records_window = tk.Toplevel(window)
    records_window.title("All Records")
    records_window.geometry("800x400")
    records_window.configure(bg="lightgray")

    tree = ttk.Treeview(records_window, columns=("PID", "Full Name", "Sex", "Address", "Contact", "Patient Details"), show="headings")
    tree.heading("PID", text="PID")
    tree.heading("Full Name", text="Full Name")
    tree.heading("Sex", text="Sex")
    tree.heading("Address", text="Address")
    tree.heading("Contact", text="Contact")
    tree.heading("Patient Details", text="Patient Details")
    tree.grid(column=0, row=0, padx=20, pady=20)

    sort_by_id_btn = tk.Button(records_window, text="Sort by PID", command=sort_records_by_id)
    sort_by_id_btn.grid(column=0, row=1, padx=10, pady=5)

    for record in mycol.find():
        pid = record.get("PID", "")
        name = record.get("Name", "")
        sex = record.get("Sex", "")
        address = record.get("Address", "")
        contact = record.get("Contact", "")
        patient_details = record.get("Patient Details", "")
        tree.insert('', 'end', values=(pid, name, sex, address, contact, patient_details))

def insert_hyphen(event):
    phone = cContact.get()
    
    # Remove any non-digit characters except hyphens
    phone = re.sub(r'\D', '', phone)

    # Check if the phone number is already in the correct format
    if len(phone) == 10:
        formatted_phone = f"{phone[:3]}-{phone[3:6]}-{phone[6:]}"
        Contact.delete(0, tk.END)
        Contact.insert(0, formatted_phone)
        return

    # If the phone number is not in the correct format, insert hyphens
    if len(phone) == 3 and phone[-1] != '-':
        phone += "-"
    elif len(phone) == 7 and phone[-1] != '-':
        phone = phone[:3] + "-" + phone[3:] + "-"
    elif len(phone) > 10:
        phone = phone[:10]  # Limit phone number to 10 digits

    Contact.delete(0, tk.END)
    Contact.insert(0, phone)


def msgbox(msg, titlebar):
    result=messagebox.askokcancel(title=titlebar, message=msg)
    return result

def validate_Address(Address):
    # Pattern to match a typical home address format with letters, numbers, space, and underscore
    pattern = re.compile(r'^[a-zA-Z0-9\s_]+, [a-zA-Z\s]+\s+\d{5}(?:-\d{4})?$')
    return bool(pattern.match(Address))

def validate_name(name):
    pattern = re.compile(r'^[a-zA-Z\s]+$')
    return bool(pattern.match(name))

def validate_Contact_number(Contact_number):
    pattern = re.compile(r'^\d{3}-\d{3}-\d{4}$')
    return bool(pattern.match(Contact_number)) and len(Contact_number.replace("-", "")) == 10

def save():
    try:
        new_id = int(cid.get())
    except ValueError:
        messagebox.showerror("Error", "PID must be numerical.")
        return

    existing_record = mycol.find_one({"PID": new_id})
    if existing_record:
        messagebox.showerror("Error", "PID already exists. Please use a unique Patient ID.")
        return

    Contact_number = cContact.get()
    if not validate_Contact_number(Contact_number):
        messagebox.showerror("Error", "Contact number must be in the format xxx-xxx-xxxx.")
        return

    Address = cAddress.get()
    if not validate_Address(Address):
        messagebox.showerror("Error", "Invalid Address format, ex: 1234 Street Name, City 12345.")
        return

    name = cname.get()
    sex = csex.get()
    if not validate_name(name) or not validate_name(sex):
        messagebox.showerror("Error", "Full Name and Sex must contain only letters.")
        return

    existing_Address_record = mycol.find_one({"Address": Address})

    existing_Contact_record = mycol.find_one({"Contact": Contact_number})
    if existing_Contact_record:
        messagebox.showerror("Error", "Contact number already exists. Please use a unique Contact number.")
        return

    r = msgbox("Save recorded?", "Record")
    if r:
        mydict = {
            "PID": new_id,
            "Name": name,
            "Sex": sex,
            "Address": Address,
            "Contact": Contact_number,
            "Patient Details": cpdetails.get()
        }
        mycol.insert_one(mydict)
        clear_fields()
        print("Record successfully saved.")

def delete():
    def confirm_delete():
        r = msgbox("Are you sure you want to delete this record?", "Confirm Delete")
        if r:
            target_id = int(cid.get())
            existing_record = mycol.find_one({"PID": target_id})
            if existing_record:
                mycol.delete_one({"PID": target_id})
                clear_fields()
                print("Record successfully deleted.")
            else:
                messagebox.showerror("Error", "ID does not exist.")
    
    confirm_delete()

def update():
    target_id = int(cid.get())
    existing_record = mycol.find_one({"PID": target_id})
    if existing_record:
        r = msgbox("Update?", "Record")
        if r:
            name = cname.get()
            sex = csex.get()

            if sex and not sex.isalpha():
                messagebox.showerror("Error", "Sex must contain only letters.")
                return

            new_values = {}

            if name and name != existing_record.get("Name"):
                new_values["Name"] = name
            if sex and sex != existing_record.get("Sex"):
                new_values["Sex"] = sex
            
            if cAddress.get():
                Address = cAddress.get()
                if not validate_Address(Address):
                    messagebox.showerror("Error", "Invalid Address format, ex: 1234 Street Name, City 12345.")
                    return 

                existing_Address_record = mycol.find_one({"Address": Address})

                new_values["Address"] = Address
            if cContact.get():
                Contact_number = cContact.get()
                if not validate_Contact_number(Contact_number):
                    messagebox.showerror("Error", "Contact number must be in the format xxx-xxx-xxxx.")
                    return  

                existing_Contact_record = mycol.find_one({"Contact": Contact_number})
                if existing_Contact_record and existing_Contact_record["_id"] != existing_record["_id"]:
                    messagebox.showerror("Error", "Contact number already exists. Please use a unique Contact number.")
                    return 

                new_values["Contact"] = Contact_number

            if cpdetails.get():
                new_values["Patient Details"] = cpdetails.get()

            if new_values:
                mycol.update_one({"PID": target_id}, {"$set": new_values})
                clear_fields()
                print("Record successfully updated.")
            else:
                print("No changes to update.")
    else:
        messagebox.showerror("Error", "PID does not exist.")

def search():
    target_id = int(cid.get())
    existing_record = mycol.find_one({"PID": target_id})
    if existing_record:
        cid.set(existing_record.get("PID", ""))
        cname.set(existing_record.get("Name", ""))
        csex.set(existing_record.get("Sex", ""))
        cAddress.set(existing_record.get("Address", ""))
        cContact.set(existing_record.get("Contact", ""))
        cpdetails.set(existing_record.get("Patient Details", ""))
    else:
        messagebox.showerror("Error", "Patient ID does not exist.")

def clear_fields():
    cid.set("")
    cname.set("")
    csex.set("")
    cAddress.set("")
    cContact.set("")
    cpdetails.set("")

def clear_all_fields():
    cid.set("")
    cname.set("")
    csex.set("")
    cAddress.set("")
    cContact.set("")
    cpdetails.set("")

window = tk.Tk()
window.title("Pharmacy Database")
window.geometry("600x300")
window.configure(bg="orange")

label = tk.Label(window, text="Pharmacy Database", width=30, height=1, bg="yellow", anchor="center")
label.config(font=("Courier", 10))
label.grid(column=1, row=1, columnspan=2)

label = tk.Label(window, text="Patient ID:", width=10, height=1, bg="yellow")
label.grid(column=1, row=2)
cid = tk.StringVar()
patientid = tk.Entry(window, textvariable=cid)
patientid.grid(column=2, row=2)

label = tk.Label(window, text="Full Name:", width=10, height=1, bg="yellow")
label.grid(column=1, row=3)
cname = tk.StringVar()
patientname = tk.Entry(window, textvariable=cname)
patientname.grid(column=2, row=3)

label = tk.Label(window, text="Sex:", width=10, height=1, bg="yellow")
label.grid(column=1, row=4)
csex = tk.StringVar()
patientsex = tk.Entry(window, textvariable=csex)
patientsex.grid(column=2, row=4)

label = tk.Label(window, text="Address:", width=10, height=1, bg="yellow")
label.grid(column=1, row=5)
cAddress = tk.StringVar()
Address = tk.Entry(window, textvariable=cAddress)
Address.grid(column=2, row=5)

label = tk.Label(window, text="Contact:", width=10, height=1, bg="yellow")
label.grid(column=1, row=6)
cContact = tk.StringVar()
Contact = tk.Entry(window, textvariable=cContact)
Contact.grid(column=2, row=6)
Contact.bind("<KeyRelease>", lambda event: insert_hyphen(event))  

label = tk.Label(window, text="Patient Details:", width=10, height=1, bg="yellow")
label.grid(column=1, row=7)
cpdetails = tk.StringVar()
pdetails = tk.Entry(window, textvariable=cpdetails)
pdetails.grid(column=2, row=7)

image = Image.open("FIGs/pharmacy.jpg") 
image = image.resize((200, 200)) 
photo = ImageTk.PhotoImage(image)

image_label = tk.Label(window, image=photo)
image_label.image = photo 
image_label.grid(column=4, row=2, rowspan=6, padx=10)

savebtn = tk.Button(window, text="Save", command=save)
savebtn.grid(column=1, row=8)
deletebtn = tk.Button(window, text="Delete", command=delete)
deletebtn.grid(column=2, row=8)
updatebtn = tk.Button(window, text="Update", command=update)
updatebtn.grid(column=3, row=8)
searchbtn = tk.Button(window, text="Search", command=search)
searchbtn.grid(column=3, row=2)

display_all_btn = tk.Button(window, text="Display All Records", command=display_all_records)
display_all_btn.grid(column=2, row=9, columnspan=2, pady=10)

clear_all_btn = tk.Button(window, text="Clear All Fields", command=clear_all_fields)
clear_all_btn.grid(column=3, row=9, columnspan=2, pady=10)

window.mainloop()
